In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
%load_ext ipython_unittest
%load_ext ipython_nose
%load_ext ipython_pytest
from IPython.display import HTML
import termios, fcntl, struct
fcntl.ioctl(1, termios.TIOCSWINSZ, struct.pack('hhhh', 57, 120, 0, 0))  # terminal width correction
HTML('''<link rel="stylesheet" href="eniram-theme/eniram-theme.css" type="text/css"></link>
        <script type="text/javascript" src="eniram-theme/rise-shortcuts.js"></script>''')

# Test parameterization
- run the same test scenario with different inputs

- avoid re-writing test setup/execution/teardown for each case

## Challenges
- clarity of syntax

- labeling arguments

- how to identify cases from test output

## Parameterization methods and libraries
- unittest:
  - [metaclasses](http://stackoverflow.com/a/20870875/15770)
  - [load_tests](http://stackoverflow.com/a/23508426/15770)
  - [inheritance](http://bugs.python.org/msg151444)
  - [unittest.TestCase.subTest()](https://docs.python.org/3/library/unittest.html#distinguishing-test-iterations-using-subtests)
    (Python >= 3.4, unittest2)

- nose_parameterized

- @pytest.mark.parametrize()

- lots of other methods on [Stack Overflow](http://stackoverflow.com/q/32899/15770)

- testdimensions

## A simple test function:

In [ ]:
import math

def circumference(radius):
    return 2.0 * math.pi * radius

## Unittest: separate test Cases
- pretty good if no setup/teardown needed
- easy to identify test cases
- verbose

In [ ]:
from unittest import TestCase

In [ ]:
%%unittest

class CircumferenceTestCase(TestCase):
    def test_circumference_zero(self):
        self.assertEqual(circumference(0.0), 0.0)
        
    def test_circumference_half(self):
        self.assertEqual(circumference(0.5), math.pi)
        
    def test_circumference_inf(self):
        self.assertEqual(circumference(float('inf')), float('inf'))

    def test_circumference_one(self):
        self.assertEqual(circumference(1.0), 6.28)

# `TestCase.subTest()`
- the `radius=radius` argument causes `(radius=<value>)` to apper in test failures
- label arguments with e.g. tests as dicts
- boilerplate in test method

In [ ]:
%%unittest

class CircumferenceTestCase(TestCase):
    tests = [{'radius': 0.0, 'expect': 0.0}, 
             {'radius': 0.5, 'expect': math.pi}, 
             {'radius': float('inf'), 'expect': 1000000.0}]
    
    def test_circumference(self):
        for arguments in self.tests:
            with self.subTest(radius=arguments['radius']):
                self.assertEqual(circumference(arguments['radius']), arguments['expect'])

# Classless test cases
- works with Nose and Pytest

In [ ]:
%%nose -v

def test_zero_radius():
    assert circumference(0.0) == 0.0

def test_positive_radius():
    assert circumference(0.5) == math.pi

def test_infinite_radius():
    assert circumference(float('inf')) == float('inf')
    
def test_example_failure():
    assert circumference(1.0) == 6.28

## Classless: setup/teardown causes repeated code
- example: to test Pandas rolling sums, we need to set up test data
- repeated in every test case

In [ ]:
import numpy as np, pandas as pd
from numpy.testing import assert_allclose

In [ ]:
%%nose -v

def test_rolling_sum_positive():
    data = pd.Series([0.09, 0.42, 0.01, 0.57, 1.12, 1.41])
    result = data.rolling(window=3).sum()
    expect = [np.nan, np.nan, 0.52, 1.00, 1.70, 3.10]
    assert_allclose(result, expect, equal_nan=True)
    
def test_rolling_sum_with_inf():
    data = pd.Series([0.09, 0.42, np.inf, 1.12, 1.41])
    result = data.rolling(window=2).sum()
    expect = [np.nan, 0.51, np.nan, np.nan, 2.53]
    assert_allclose(result, expect, equal_nan=True)

# Parameterize by Unittest inheritance
- easy to label arguments
- tests are easy to identify in output
- can parameterize multiple tests in one test case class

In [ ]:
%%unittest

class RollingSumTestBase:                                        # the base class
    def test_rolling_sum(self):
        data = pd.Series(self.data)
        result = data.rolling(window=self.window).sum()
        assert_allclose(result, self.expect, equal_nan=True)
        
class RollingSumGteZero(RollingSumTestBase, TestCase):           # test case 1
    data = [0.10, 0.42, 0.00, 0.58, 1.12, 1.40]
    window = 3
    expect = [np.nan, np.nan, 0.52, 1.00, 1.70, 3.10]
    
class RollingSumWithInfTestCase(RollingSumTestBase, TestCase):   # test case 2
    data = [0.10, 0.42, np.inf, 1.12, 1.40]
    window = 2
    expect = [np.nan, 0.52, np.nan, np.nan, 2.52]

## Nose `yield` style parameterization
- debugger unfriendly
- can't label arguments

In [ ]:
%%nose -v

def test_rolling_sum_with_yield():
    
    def check(description, data, window, expect):
        """rolling(window={window}).sum() with {description}"""
        series = pd.Series(data)
        result = series.rolling(window=window).sum()
        assert_allclose(result, expect, equal_nan=True)
    
    yield check, 'positive floats', [0.09, 0.42, 0.01, 0.57, 1.13, 1.40], 3, [np.nan, np.nan, 0.52, 1.00, 1.71, 3.10]
    yield check, 'infinities', [0.10, 0.42, np.inf, 1.12, 1.40], 2, [np.nan, 0.52, np.nan, np.nan, 2.52]

# nose_parameterized
- a Nose helper package available on PyPI

- concise code

- can label arguments using the `param()` helper

- has ways to customize test output

## @parameterized

In [ ]:
from nose_parameterized import parameterized

In [ ]:
%%nose -v

@parameterized([
    ([0.10, 0.42, 0.00, 0.58, 1.12, 1.40], 3,
     [np.nan, np.nan, 0.52, 1.00, 1.70, 3.10]),
    ([0.10, 0.42, np.inf, 1.12, 1.40], 2,
     [np.nan, 0.52, np.nan, np.nan, 2.52])])
def test_rolling_sum_noseparameterized(data, window, expect):
    series = pd.Series(data)
    result = series.rolling(window=window).sum()
    assert_allclose(result, expect, equal_nan=True)

## `@parameterized` with keyword arguments

In [ ]:
from nose_parameterized import param

In [ ]:
%%nose -v

@parameterized([
    param(data=[0.10, 0.42, 0.00, 0.58, 1.12, 1.40], 
          window=3,
          expect=[np.nan, np.nan, 0.52, 1.00, 1.70, 3.10]),
    param(data=[0.10, 0.42, np.inf, 1.12, 1.40], 
          window=2,
          expect=[np.nan, 0.52, np.nan, np.nan, 2.52])])
def test_rolling_sum_noseparameterized(data, window, expect):
    series = pd.Series(data)
    result = series.rolling(window=window).sum()
    assert_allclose(result, expect, equal_nan=True)

## `@parameterized` custom test descriptions

- will be available nose_parameterized 0.6

In [ ]:
from nose_parameterized import param, parameterized

def rolling_sum_doc_func(func, num, param):
    return ('rolling(window={window}).sum() with {description}'
            .format(**param.kwargs))

In [ ]:
%%nose -v

@parameterized(
    [param(description='positive floats',
           data=[0.09, 0.42, 0.01, 0.57, 1.13, 1.40], 
           window=3,
           expect=[np.nan, np.nan, 0.52, 1.00, 1.71, 3.10]),
     param(description='infinities',
           data=[0.10, 0.42, np.inf, 1.12, 1.40], 
           window=2,
           expect=[np.nan, 0.52, np.nan, np.nan, 2.52])],
    doc_func=rolling_sum_doc_func)                    # <--- define the test description function to use  <---
def test_rolling_sum_noseparameterized(description, data, window, expect):
    series = pd.Series(data)
    result = series.rolling(window=window).sum()
    assert_allclose(result, expect, equal_nan=True)

## `@parameterized` with test output template
- define the test description as a string template, not a custom function

In [ ]:
from nose_parameterized import param, parameterized

def description_template(template):
    def doc_func(func, num, param):
        return template.format(**param.kwargs)
    return doc_func

In [ ]:
%%nose -v

@parameterized(
    [param(description='positive floats',
           data=[0.09, 0.42, 0.01, 0.57, 1.13, 1.40], 
           window=3,
           expect=[np.nan, np.nan, 0.52, 1.00, 1.71, 3.10]),
     param(description='infinities',
           data=[0.10, 0.42, np.inf, 1.12, 1.40], 
           window=2,
           expect=[np.nan, 0.52, np.nan, np.nan, 2.52])],
    doc_func=description_template('rolling(window={window}).sum() with {description}'))  # <--- the template <---
def test_rolling_sum_noseparameterized(description, data, window, expect):
    series = pd.Series(data)
    result = series.rolling(window=window).sum()
    assert_allclose(result, expect, equal_nan=True)

## `@parameterized` subclass with docstring template

In [ ]:
from nose_parameterized.parameterized import default_doc_func

class parameterized_plus_description(parameterized):
    @staticmethod
    def _template_doc_func(func, num, param):                   # the template helper method
        if func.__doc__: return func.__doc__.format(**param.kwargs)
        else: return default_doc_func(func, num, param)    
    
    def __init__(self, input):                                  # override @parameterized()
        super().__init__(input, doc_func=self._template_doc_func)
        
    @classmethod
    def expand(cls, input, **kwargs):                           # override @parameterized.expand()
        return super().expand(input, doc_func=cls._template_doc_func, **kwargs)

In [ ]:
%%nose -v

@parameterized_plus_description(                                #  @parameterized_plus_description() instead of @parameterized()
    [param(description='positive floats',
           data=[0.09, 0.42, 0.01, 0.57, 1.13, 1.40], 
           window=3,
           expect=[np.nan, np.nan, 0.52, 1.00, 1.71, 3.10]),
     param(description='infinities',
           data=[0.10, 0.42, np.inf, 1.12, 1.40], 
           window=2,
           expect=[np.nan, 0.52, np.nan, np.nan, 2.52])])
def test_rolling_sum_noseparameterized(description, data, window, expect):
    """rolling(window={window}).sum() with {description}"""     # test description template as docstring
    series = pd.Series(data)
    result = series.rolling(window=window).sum()
    assert_allclose(result, expect, equal_nan=True)

## @pytest.mark.parametrize
- the built-in parameterization solution in pytest

- must enumerate argument names

- can't use keywords to label arguments

- can't give verbose descriptions for tests

## @pytest.mark.parametrize: example

In [ ]:
%%pytest -v

import numpy as np
from numpy.testing import assert_allclose
import pandas as pd
import pytest

@pytest.mark.parametrize(
    'data,window,expect',                           # must enumerate argument names
    [([0.10, 0.42, 0.00, 0.58, 1.12, 1.40], 3,
      [np.nan, np.nan, 0.52, 1.00, 1.70, 3.10]),
     ([0.10, 0.42, np.inf, 1.12, 1.40], 2,          # can't label arguments
      [np.nan, 0.52, np.nan, np.nan, 2.52])],
    ids=['positive floats', 'infinities'])          # label tests as a separate list
def test_rolling_sum_pytest(data, window, expect):
    series = pd.Series(data)
    result = series.rolling(window=window).sum()
    assert_allclose(result, expect, equal_nan=True)

## @pytest.mark.parametrize: test description as an argument
- function name and all arguments still visible

In [ ]:
%%pytest -v

import numpy as np
from numpy.testing import assert_allclose
import pandas as pd
import pytest

@pytest.mark.parametrize(
    'description,data,window,expect',
    [('positive floats',                           # test description as an argument 
      [0.10, 0.42, 0.00, 0.58, 1.12, 1.40], 3,
      [np.nan, np.nan, 0.52, 1.00, 1.70, 3.10]),
     ('infinities',                                # test description as an argument
      [0.10, 0.42, np.inf, 1.12, 1.40], 2,
      [np.nan, 0.52, np.nan, np.nan, 2.52])])
def test_rolling_sum_pytest(description, data, window, expect):
    series = pd.Series(data)
    result = series.rolling(window=window).sum()
    assert_allclose(result, expect, equal_nan=True)

## Multi-dimensional tests with `testdimensions`

In [ ]:
%%pytest -v --tb=short

import pandas as pd, numpy as np, pytest
from testdimensions import pytest_mark_dimensions

@pytest_mark_dimensions(
    'values,index,method,order,expect', """
    index = [0, 1, 2, 3, 4, 5]
    
                            1            2            3
    'spline'      [4.32,6.16]  [3.82,5.68]  [3.70,5.80]
    'polynomial'  [4.00,6.00]  [3.75,5.75]  [3.70,5.80]

    index = [0, 1, 3, 5, 8, 13]
    
                            1            2            3
    'spline'      [3.47,4.89]  [4.13,5.88]  [3.94,5.74]
    'polynomial'  [3.71,5.43]  [3.91,5.66]  [3.94,5.74] """,
    values=[1.0, 2.0, np.nan, np.nan, 8.0, 10.0],
    nan=np.nan)
def test_interpolate(values, index, method, order, expect):
    series = pd.Series(values, index=index)
    result = series.interpolate(method=method, order=order)
    expect_series = pd.Series(expect)
    np.testing.assert_allclose(result[2:4], expect_series, equal_nan=True, atol=1e-2)

In [ ]:
%%pytest -v --tb=short

import pandas as pd, numpy as np, pytest
from testdimensions import pytest_mark_dimensions

@pytest_mark_dimensions(
    'values,index,method,order,expect', """
    index = [0, 1, 2, 3, 4, 5]
    
                            1            2            3
    'spline'      [4.32,6.16]  [3.82,5.68]  [3.70,5.80]
    'polynomial'  [4.00,6.00]  [3.75,5.75]  [3.70,5.80]

    index = [0, 1, 3, 5, 8, 13]
    
                            1            2            3
    'spline'      [3.47,4.89]  [4.13,5.88]  [3.94,5.74]
    'polynomial'  [3.71,5.43]  [3.91,5.66]  [3.94,5.74] """,
    values=[1.0, 2.0, np.nan, np.nan, 8.0, 10.0],
    nan=np.nan)
def test_intersection(values, index, method, order, expect):
    series = pd.Series(values, index=index)
    result = series.interpolate(method=method, order=order)
    expect_series = pd.Series(expect)
    np.testing.assert_allclose(result[2:4], expect_series, equal_nan=True, atol=1e-2)

# Open Source from Eniram

https://github.com/EniramLtd/testdimensions

<img src="testdimensions.github.png" />